### fill missing value of aq

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
info_df = pd.read_csv("./data/lose_hour_record.csv")
aq_df = pd.read_csv("./data/air_quality_concated.csv")

# info_df[(info_df["pollutant"] == "PM2.5") & (info_df["counts"] >= 3000)].index.tolist()
# info_df[info_df["counts"] == 3][1000:1002]
# info_df.iloc[0].index
fill_threshold = 6
info_df.counts.unique()

In [ ]:
# info_df:['stationId', 'pollutant', 'start_time', 'counts']
# aq_df:stationId time PM2.5  PM10 NO2 CO O3 SO2 

new_info_df = info_df[info_df["counts"] <= fill_threshold]
print(new_info_df.shape[0])
pollutants = ["PM2.5", "PM10", "NO2", "CO", "O3" "SO2"]

d_format = "%Y-%m-%d %H:%M:%S"
# first filter counts <= threshold 

def shift_time(date_str, shift):
    new_date = datetime.strptime(date_str, d_format) + pd.Timedelta("{} hour".format(str(shift)))
    new_str = datetime.strftime(new_date, d_format)
    return new_str

def fill_na_values(i_df):
    counts = i_df["counts"]
    
    start_time = i_df["start_time"]
    # check whether continuous na on the bottom

    last_not_na_time = shift_time(start_time,-1)
    next_not_na_time = shift_time(start_time,counts)

    pollutant = i_df["pollutant"]

    last_df = aq_df[(aq_df["stationId"] == i_df["stationId"]) & \
                       (aq_df["time"] == last_not_na_time)]

    start_index = last_df.index.tolist()[0] + 1

    next_df = aq_df[(aq_df["stationId"] == i_df["stationId"]) & \
                       (aq_df["time"] == next_not_na_time)]

    last_value = last_df[pollutant].iloc[0]
    next_value = next_df[pollutant].iloc[0]

    interval = (next_value-last_value) / (counts+1)

    for i in range(counts):
#             aq_df.iloc[i+start_index][pollutant] = (last_value + (i+1)*interval)
        aq_df.set_value((i+start_index),pollutant,(last_value + (i+1)*interval))

    # what if next_not_na_time out of bound?
        
        
new_info_df = new_info_df.iloc[20100:20200]
new_info_df.apply(fill_na_values,axis=1)

In [ ]:
aq_df = aq_df.interpolate(limit=3)

### Fill grid weather's na

In [32]:
import pandas as pd
import numpy as np
from datetime import datetime
data_path = "../MSBD5002PROJECT_data/"

In [12]:
# load air quality file and grid weather file
aq_df = pd.read_csv("../MSBD5002PROJECT_data/air_quality_concated.csv")
grid_df = pd.read_csv("../MSBD5002PROJECT_data/grid_weather_1701_1804_unfill.csv")
# grid_4_df = pd.read_csv("../MSBD5002PROJECT_data/gridWeather_201804.csv")

In [11]:
def time_range(str_start='2018-04-01 00:00:00',str_end='2018-04-30 23:00:00'):
    timeFrom = datetime.strptime(str_start, "%Y-%m-%d %H:%M:%S")
    timeTo = datetime.strptime(str_end,"%Y-%m-%d %H:%M:%S")
    Hours_Delta = pd.date_range(timeFrom, timeTo, freq='H').strftime('%Y-%m-%d %H:%M:%S')
    
    return Hours_Delta

In [59]:
aq_df.time.unique()
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7498182 entries, 0 to 7498181
Data columns (total 7 columns):
humidity          float64
pressure          float64
station_id        object
temperature       float64
time              object
wind_direction    float64
wind_speed        float64
dtypes: float64(5), object(2)
memory usage: 400.4+ MB


In [21]:
apr_grid_df = grid_df[grid_df["time"] >= "2018-04-01 00:00:00"]
station_rows = {}

for grid in grid_df.station_id.unique():
    each_station = apr_grid_df[apr_grid_df["station_id"] == grid].shape[0]
    station_rows[grid] = each_station

# missing_time_grid_stations = [i for i in station_rows.keys() if station_rows[i] != 712]

In [25]:
Times_missed = {}
grid_items = ['humidity', 'pressure', 'temperature', 'wind_direction', 'wind_speed']
Stations = apr_grid_df.station_id.unique()
for station in Stations:
    Times_missed[station] = {}
    # Find time not in rows.
    tmp = apr_grid_df[apr_grid_df['station_id']==station].time.unique()
    tmp = list(set(time_range()) - set(tmp))
    
    tmp_df = pd.DataFrame(tmp, columns=['time'])
    tmp_df['station_id'] = station
    for item in grid_items:
        tmp_df[item] = np.nan
    tmp_df = tmp_df[apr_grid_df.columns]
    apr_grid_df = pd.concat([apr_grid_df, tmp_df], ignore_index=True)

In [33]:
apr_grid_df = apr_grid_df.sort_values(by=['station_id','time'])
apr_grid_df = apr_grid_df.reset_index(drop=True)
apr_grid_df.to_csv(data_path+'grid_04_timestamp_filled.csv',index=None)

In [83]:
a = pd.read_csv(data_path+"grid_04_timestamp_filled.csv",index_col=None)

In [101]:
a.to_csv(data_path+'grid_04_timestamp_filled.csv',index=None)

In [51]:
station_rows = {}
for grid in a.station_id.unique():
    each_station = a[a["station_id"] == grid].shape[0]
    station_rows[grid] = each_station


dup_time_grid_stations = [i for i in station_rows.keys() if station_rows[i] > 720]
dup_time_grid_stations

[]

In [85]:
a = a.interpolate(limit=3)

In [76]:
a.isna().any()

humidity          False
pressure          False
station_id        False
temperature       False
time              False
wind_direction    False
wind_speed        False
dtype: bool

In [117]:
a.to_csv(data_path+'grid_04_all_filled.csv',index=None)